## Библиотека LightGBM

Инструкция по запуску: каждая ячейка одну за одной. Ничего лишнего или недостающего нет.

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook as tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

%matplotlib inline

Для label кодирования признаков, отвечающих за классы услуги

In [2]:
def encode_option(a):
    if a == 'econom':
        return 1
    if a == 'business':
        return 2
    if a == 'vip':
        return 3
    else:
        return 0

# Обработаем данные
Добавим признаки со временем и датой, сумму по классам и самое важное количество заказов в час, в радиусе 120 метров, в одну дату

Лишнее удаляем

In [3]:
train = pd.read_csv('train_data.csv')
target = pd.read_csv('train_target.csv')
test = pd.read_csv('test_data.csv')

train['target'] = target

In [4]:
train.head()

,dist,due,f_class,lat,lon,s_class,t_class,target
0,5117.239228,2014-01-22 04:10:00.000,econom,55.669290,37.474336,business,NaN,0
1,3638.726690,2014-01-10 13:45:00.000,econom,55.807461,37.635045,business,NaN,0
2,15143.069693,2014-03-02 02:10:00.000,econom,55.741105,37.615821,NaN,NaN,1
3,-1.000000,2014-02-26 13:20:00.000,econom,55.625526,37.618543,NaN,NaN,1
4,4708.142572,2014-01-21 19:45:00.000,business,55.813167,37.597863,NaN,NaN,0


In [4]:
array = []
for data in tqdm_notebook((train, test)):
    
    data.loc[:,'time_as_str'] = data['due'].apply(lambda x: x[x.find(' ') + 1:-4])
    data.loc[:, 'time_in_seconds'] = pd.to_timedelta(data['time_as_str']).dt.total_seconds()
    
    #признаки со временем и датой: дата, час, минуты, месяц, день недели кодированный
    
    data['due'] = pd.to_datetime(data['due'])
    data['date'] = data['due'].dt.date
    data['hour'] = data['due'].dt.hour
    data['weekday'] = data['due'].dt.weekday
    
    for method in ('date', 'hour','weekday', 'minute', 'month'):
        data[method] = getattr(data['due'].dt, method)
        
    #сложим признаки, отвечающие за классы
    
    for option in ('f', 's', 't'):
        data[option] = data.loc[:, option + '_class'].apply(encode_option)
    data['sum'] = data['f'] + data['s'] + data['t']

    array.append(data.copy(deep = True))


In [5]:
arr =[]
#важный признак, количество заказов в одну дату в круге радиусом 110 метро(округление до 2 знаков лат и лон) в час
train, test = array[0], array[1]
for data in (train, test):
    for coords in ('lat', 'lon'):
        data[coords + '_round'] = round(data[coords],3)
    data['index'] = data.index 
    
    df = data.groupby(['lat_round','lon_round', 'date', 'hour'], as_index = False).agg({'index': 'count'})
    df = pd.DataFrame(df)
    data_ = pd.merge(data, df,  how='left', left_on=['lat_round','lon_round', 'date', 'hour'],
                      right_on = ['lat_round','lon_round', 'date', 'hour'])
    
    data_.drop(['date','time_as_str','hour', 'due',
               't', 'f', 's', 'f_class', 's_class', 't_class'], axis=1, inplace=True)
    
    arr.append(data_.copy(deep = True))

train, test = arr[0], arr[1]

In [6]:
test.columns

Index(['dist', 'lat', 'lon', 'time_in_seconds', 'weekday', 'minute', 'month',
       'sum', 'lat_round', 'lon_round', 'index_x', 'index_y'],
      dtype='object')

# Кластеризация 
кластеризуем и добавим признаки расстояние до центра кластера и номер кластера

In [7]:
def clust(train, test):
    model = KMeans(n_clusters=5, n_jobs=-1)
    model.fit(train[['lat', 'lon']])
    
    for data in (train, test):
        data['clust'] =  model.predict(data[['lat', 'lon']])
        data['dist_clust'] = model.transform(data[['lat', 'lon']]).min(axis=1)
    
    plt.scatter(test.loc[:, 'lon'], test.loc[:, 'lat'], c=test['clust'], cmap='viridis')
    centers = model.cluster_centers_
    plt.scatter(centers[:, 1], centers[:, 0], c='black', alpha=0.5)


In [ ]:
clust(train, test)

In [ ]:
feature_columns = train.columns.tolist()
feature_columns.pop(feature_columns.index('target'))

target_column = ['target']

# Модель

In [ ]:
train__, val__ = train_test_split(train, test_size = 0.3)

In [ ]:
train__

In [ ]:
from lightgbm import LGBMClassifier 

clf = LGBMClassifier(n_estimators=100, learning_rate=0.05, num_leaves=63)
clf.fit(
    train__[feature_columns], train__[target_column].values.ravel(),
    eval_set=[(val__[feature_columns], val__[target_column].values.ravel())],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=200,
)

## Обучение на всем трейне

In [ ]:
from lightgbm import LGBMClassifier 

clf = LGBMClassifier(n_estimators=1200, learning_rate=0.05, num_leaves=63)
clf.fit(train[feature_columns], train[target_column].values.ravel())

In [ ]:
sorted(zip(clf.feature_importances_, feature_columns))

In [ ]:
predict = clf.predict_proba(test[feature_columns])

In [ ]:
predict.shape

In [ ]:
pd.DataFrame(data={"target": predict[:, 1]}, index=test.index).to_csv("10.csv", index_label='index')

### На валидации: 0.730788
### На тест из Kaggle: 0.73439